In [37]:
import h5py
import numpy as np
import scipy.io
import matlab.engine

In [12]:
eng = matlab.engine.start_matlab()

In [46]:
h5_path = '../data/easycom/Easycom_N-1020_fs-48k_nrfft-1025.h5'

ind_mics = [0,2]
n_mics = len(ind_mics)
f = h5py.File(h5_path,'r')
fs = int(f["fs"][()])
doas = np.array(f['doas']) # (ndarray) [n_az x n_el x 2]
airs = np.array(f['airs'])[:,:,ind_mics,:]       # (ndarray) [n_az x n_el x n_chan x n_smpl]
atfs = np.array(f['atfs'])[...,ind_mics]         # (ndarray) [n_az x n_el x n_rfft x n_chan]
rtfs = np.array(f['rtfs'])[...,ind_mics,ind_mics] # (ndarray) [n_az x n_el x n_rfft x n_chan x n_chan]
mic_pos = np.array(f['mic_pos'])[:,ind_mics]     # (ndarray) [3 x n_chan]
f.close()

In [47]:
_doas = doas.reshape(-1,2)
print(_doas.shape)
n_obs = _doas.shape[0]


data = airs
n_smpl = data.shape[-1]
print(data.shape)
data = data.reshape(n_obs, n_mics, n_smpl)
print(data.shape)


azimuths = []
hrtfs = []

for n in range(n_obs):

    if not _doas[n,0] == 0:
        continue

    azimuths.append(_doas[n,:])
    hrtfs.append(data[n,:])


azimuths = np.array(azimuths)
hrtfData = np.stack(hrtfs, axis=0)
print(azimuths.shape)
print(hrtfData.shape)

sourcePosition = np.rad2deg(azimuths)
desiredPosition = np.stack(np.meshgrid(
    np.arange(0.,360.,3),
    np.array([0])
), axis=-1).reshape(-1,2)
print(desiredPosition.shape)

to_save = {
    'desiredPosition' : np.float64(desiredPosition),
    'hrtfData' : np.float64(hrtfData),
    'sourcePosition' : np.float64(sourcePosition),
}

scipy.io.savemat('../data/easycom.mat', to_save)


(1020, 2)
(60, 17, 2, 768)
(1020, 2, 768)
(17, 2)
(17, 2, 768)
(120, 2)


Error using interpolateHRTF
Not enough input arguments.



MatlabExecutionError: 
  File /usr/local/MATLAB/R2022b/toolbox/audio/audio/interpolateHRTF.m, line 70, in interpolateHRTF
Not enough input arguments.
